In [ ]:
import os
import csv
import pandas as pd

df1 = pd.read_csv('datasets/players.csv')
df2 = pd.read_csv('datasets/players_teams.csv')
df3 = pd.read_csv('datasets/awards_players.csv')

players_teams_file = df2.drop(columns=['lgID'])
players_file = df1[df1['pos'].notna() & (df1['pos'] != '')]
players_file = players_file.drop(columns=['firstseason', 'lastseason', 'deathDate'])
merged_df = pd.merge(players_teams_file, players_file, left_on='playerID', right_on='bioID', how='left')
merged_df = merged_df.drop(columns=['bioID'])
awards_players_file = df3.drop(columns=['lgID'])

awards_grouped = awards_players_file.groupby(['playerID', 'year'])['award'].apply(list).reset_index()

merged_df = pd.merge(merged_df, awards_grouped, on=['playerID', 'year'], how='left')

print(merged_df)
merged_df.to_csv('oopsie.csv', index=False)
print(f"Dataset guardado em: {'oopsie'}")